# Optimize distance : forecasting of Lorenz data as a function of horizon and output variable

This notebook accompanies the following publication:
Paul Platzer, Arthur Avenas, Bertrand Chapron, Lucas Drumetz, Alexis Mouche, Léo Vinour. Distance Learning for Analog Methods. 2024. [⟨hal-04841334⟩](https://hal.science/hal-04841334)

It is used to run optimization algorithms for numerical experiments with Lorenz system data. In particular, we investigate the dependency of the optimal distance with the catalog size (i.e. the number of available data).

This notebook required loading a lot of data into memory. It was initially run on a laptop with 32GB of RAM. Running this notebook on your device might require minor modifications of the code.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from tqdm.notebook import tqdm
from sklearn.neighbors import NearestNeighbors
import sys
sys.path.append('../../functions/.')
from analogs import apply_transform, find_analogues, compute_weights, compute_diffs, compute_mae_mad, compute_error
from distance_learning import compute_gradient_MSE, compute_regularization, learn_distance

In [2]:
data_folder = '../../data/lorenz/'
output_folder = '../../output/lorenz/'

# Load catalog

In [3]:
traj_norm = []; stds = []
for j in range(10):
    npzfile = np.load(data_folder + 'catalog_large_'+str(j)+'.npz')
    traj_norm.append(npzfile['traj_norm'])
    stds.append(npzfile['stds'])
    dt = npzfile['dt']
    Ntrain = npzfile['Ntrain']
    tau = npzfile['tau']
    h_max = npzfile['h_max']
traj_norm = np.array(traj_norm)
stds = np.array(stds)

# Optimization: loop on catalog size

In [4]:
# Define training explanatory variable and forecast horizon
train_x = traj_norm[0][::int(tau/dt)][:Ntrain]
for j in range(1,10):
    train_x = np.concatenate((train_x,traj_norm[j][::int(tau/dt)][:Ntrain]), axis=0)
hh_ind = 32
horizon = dt * hh_ind
train_y = traj_norm[0][hh_ind::int(tau/dt)][:Ntrain]
for j in range(1,10):
    train_y = np.concatenate((train_y,traj_norm[j][hh_ind::int(tau/dt)][:Ntrain]), axis=0)

del traj_norm

In [14]:
# Set parameters for optimization
k = 200
nn_algo = 'kd_tree'
loo = True
corr_length_train = 0
n_epoch = 60
learning_rate_factor = 50
regul_coef = [0]
transform_matrix = np.eye(3)

### Choose sub-catalog sizes and number of permutations to make
Ncats = np.round( len(train_x) / 10**np.linspace(4,1.5,25) ).astype(int)
Nperm = 10

# Initialize transform matrices to be stored
transform_optim = np.empty((len(Ncats), Nperm, 3, 3))
mse_optim = np.empty((len(Ncats), Nperm, n_epoch+1))

for i_ncat in tqdm(range(len(Ncats))):
    
    ncat = Ncats[i_ncat]
    ind_subcat_noperm = np.arange( 0 , len(train_x) , int(len(train_x)/(ncat*Nperm)) )
    # Set permutation for sub catalogs
    rs = np.random.RandomState(1312)
    permutation = rs.permutation(ind_subcat_noperm)
    
    for j_perm in range(Nperm):
        # Take subsample of catalog of size Ncats[i_sub] from permutation
        ind_subcat_perm = permutation[j_perm*ncat:(j_perm+1)*ncat]

        train_x_sub = train_x[ind_subcat_perm]
        train_y_sub = train_y[ind_subcat_perm]
        
        Itrain = np.arange(len(ind_subcat_perm))

        train_X_sub = apply_transform(train_x_sub, transform_matrix, Itrain)
                
        nn = NearestNeighbors( algorithm = nn_algo , 
                                  n_neighbors = k + 1 + 2*corr_length_train ) # leave-one-out procedure + anticipating time-correlated data
        nn.fit(train_X_sub)
        
        mse_init = compute_error(train_X_sub, train_y_sub, Itrain, Itrain, k, nn, loo=True, corr_length_train=0, vector_out=False, error_type='MSE')
        
        learning_rate = learning_rate_factor / mse_init
        
        result = learn_distance(train_x_sub, train_y_sub, transform_matrix, Itrain, Itrain, [], 
                           k = k, nn_algo='auto', error_type='MSE', n_epoch=n_epoch,
                            learning_rate = learning_rate, regul_coef = regul_coef,
                           loo=True, corr_length_train=corr_length_train,
                             batch_size = len(train_X_sub),
                                 verbose_batch = True)

        transform_optim[i_ncat, j_perm] = result[0][-1].copy()
        mse_optim[i_ncat, j_perm] = result[2].copy()

  0%|          | 0/25 [00:00<?, ?it/s]

Starting distance-learning algorithm with the following parameters:
Error type = MSE
Transformation type = matrix (general linear transformation)
Number of analogues = 200
Learning rate = 45.47589682008376
Number of Epochs = 60
Mini-batch size = 1000
Regularization = [0]


  0%|          | 0/60 [00:00<?, ?it/s]

epoch 1/60   |   iter. 1/1 : 
MSE(batch) = 1.0994 ;  MSE(train) = 1.0994 ;  MSE(test) = [nan]
epoch 2/60   |   iter. 1/1 : 
MSE(batch) = 0.7965 ;  MSE(train) = 0.7965 ;  MSE(test) = [nan]
epoch 3/60   |   iter. 1/1 : 
MSE(batch) = 0.0281 ;  MSE(train) = 0.0281 ;  MSE(test) = [nan]
epoch 4/60   |   iter. 1/1 : 
MSE(batch) = 0.0280 ;  MSE(train) = 0.0280 ;  MSE(test) = [nan]
epoch 5/60   |   iter. 1/1 : 
MSE(batch) = 0.0280 ;  MSE(train) = 0.0280 ;  MSE(test) = [nan]
epoch 6/60   |   iter. 1/1 : 
MSE(batch) = 0.0279 ;  MSE(train) = 0.0279 ;  MSE(test) = [nan]
epoch 7/60   |   iter. 1/1 : 
MSE(batch) = 0.0278 ;  MSE(train) = 0.0278 ;  MSE(test) = [nan]
epoch 8/60   |   iter. 1/1 : 
MSE(batch) = 0.0278 ;  MSE(train) = 0.0278 ;  MSE(test) = [nan]
epoch 9/60   |   iter. 1/1 : 
MSE(batch) = 0.0277 ;  MSE(train) = 0.0277 ;  MSE(test) = [nan]
epoch 10/60   |   iter. 1/1 : 
MSE(batch) = 0.0277 ;  MSE(train) = 0.0277 ;  MSE(test) = [nan]
epoch 11/60   |   iter. 1/1 : 
MSE(batch) = 0.0276 ;  MSE(t

  0%|          | 0/60 [00:00<?, ?it/s]

...

  0%|          | 0/60 [00:00<?, ?it/s]

epoch 1/60   |   iter. 1/1 : 
MSE(batch) = 0.0284 ;  MSE(train) = 0.0284 ;  MSE(test) = [nan]
epoch 2/60   |   iter. 1/1 : 
MSE(batch) = 0.0171 ;  MSE(train) = 0.0171 ;  MSE(test) = [nan]
epoch 3/60   |   iter. 1/1 : 
MSE(batch) = 0.0228 ;  MSE(train) = 0.0228 ;  MSE(test) = [nan]
epoch 4/60   |   iter. 1/1 : 
MSE(batch) = 0.0098 ;  MSE(train) = 0.0098 ;  MSE(test) = [nan]
epoch 5/60   |   iter. 1/1 : 
MSE(batch) = 0.0081 ;  MSE(train) = 0.0081 ;  MSE(test) = [nan]
epoch 6/60   |   iter. 1/1 : 
MSE(batch) = 0.0060 ;  MSE(train) = 0.0060 ;  MSE(test) = [nan]
epoch 7/60   |   iter. 1/1 : 
MSE(batch) = 0.0050 ;  MSE(train) = 0.0050 ;  MSE(test) = [nan]
epoch 8/60   |   iter. 1/1 : 
MSE(batch) = 0.0044 ;  MSE(train) = 0.0044 ;  MSE(test) = [nan]
epoch 9/60   |   iter. 1/1 : 
MSE(batch) = 0.0039 ;  MSE(train) = 0.0039 ;  MSE(test) = [nan]
epoch 10/60   |   iter. 1/1 : 
MSE(batch) = 0.0035 ;  MSE(train) = 0.0035 ;  MSE(test) = [nan]
epoch 11/60   |   iter. 1/1 : 
MSE(batch) = 0.0033 ;  MSE(t

  0%|          | 0/60 [00:00<?, ?it/s]

epoch 1/60   |   iter. 1/1 : 
MSE(batch) = 0.0298 ;  MSE(train) = 0.0298 ;  MSE(test) = [nan]
epoch 2/60   |   iter. 1/1 : 
MSE(batch) = 0.0183 ;  MSE(train) = 0.0183 ;  MSE(test) = [nan]
epoch 3/60   |   iter. 1/1 : 
MSE(batch) = 0.0232 ;  MSE(train) = 0.0232 ;  MSE(test) = [nan]
epoch 4/60   |   iter. 1/1 : 
MSE(batch) = 0.0096 ;  MSE(train) = 0.0096 ;  MSE(test) = [nan]
epoch 5/60   |   iter. 1/1 : 
MSE(batch) = 0.0078 ;  MSE(train) = 0.0078 ;  MSE(test) = [nan]
epoch 6/60   |   iter. 1/1 : 
MSE(batch) = 0.0061 ;  MSE(train) = 0.0061 ;  MSE(test) = [nan]
epoch 7/60   |   iter. 1/1 : 
MSE(batch) = 0.0051 ;  MSE(train) = 0.0051 ;  MSE(test) = [nan]
epoch 8/60   |   iter. 1/1 : 
MSE(batch) = 0.0045 ;  MSE(train) = 0.0045 ;  MSE(test) = [nan]
epoch 9/60   |   iter. 1/1 : 
MSE(batch) = 0.0040 ;  MSE(train) = 0.0040 ;  MSE(test) = [nan]
epoch 10/60   |   iter. 1/1 : 
MSE(batch) = 0.0037 ;  MSE(train) = 0.0037 ;  MSE(test) = [nan]
epoch 11/60   |   iter. 1/1 : 
MSE(batch) = 0.0034 ;  MSE(t

KeyboardInterrupt: 

In [11]:
# Store
np.savez(output_folder + 'optim_lorenz_catsize.npz', 
        transform_optim = transform_optim,
        mse_optim = mse_optim,
        k = k,
        learning_rate_factor = learning_rate_factor,
        regul = regul,
        n_iter = n_iter,
        )